# Create Hugging Face Dataset

<a target="_blank" href="https://colab.research.google.com/github/simonguest/CS-394/blob/main/src/07/notebooks/create-dataset.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a target="_blank" href="https://github.com/simonguest/CS-394/raw/refs/heads/main/src/07/notebooks/create-dataset.ipynb">
  <img src="https://img.shields.io/badge/Download_.ipynb-blue" alt="Download .ipynb"/>
</a>

## Install dependencies

In [5]:
!uv pip install datasets

Using Python 3.12.12 environment at: /usr
Audited 1 package in 54ms


## Configuration

**Note**: If you are running this notebook on Colab, be sure to first upload your training data (.jsonl) files.

In [6]:
TRAIN_FILE = "./sample_data/train.jsonl"
VALIDATION_FILE = "./sample_data/validation.jsonl"
TEST_FILE = "./sample_data/test.jsonl"

DATASET_REPO = "juvereturn/bible-dataset"

## Create dataset functions

In [7]:
from datasets import Dataset, DatasetDict
import json


def load_jsonl(file_path):
    data = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            if not line:  # Skip empty lines
                continue
            try:
                # Try parsing the line
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Warning: Error parsing line {line_num}: {e}")
                print(f"Problematic line: {line[:200]}...")
    return data


def create_hf_dataset(train_file, val_file, test_file):
    # Load the data
    train_data = load_jsonl(train_file)
    val_data = load_jsonl(val_file)
    test_data = load_jsonl(test_file)

    # Create datasets
    train_dataset = Dataset.from_list(train_data)
    val_dataset = Dataset.from_list(val_data)
    test_dataset = Dataset.from_list(test_data)

    # Combine into DatasetDict
    dataset_dict = DatasetDict(
        {
            "train": train_dataset,
            "validation": val_dataset,
            "test": test_dataset,
        }
    )

    return dataset_dict


# Create and validate the dataset is ready to upload
dataset = create_hf_dataset(TRAIN_FILE, VALIDATION_FILE, TEST_FILE)
print(f"Train samples: {len(dataset['train'])}")
print(f"Validation samples: {len(dataset['validation'])}")
print(f"Test samples: {len(dataset['test'])}")
print(f"\nSample entry: {dataset['train'][0]}")

Train samples: 996
Validation samples: 99
Test samples: 10

Sample entry: {'messages': [{'content': "I've been hurt by someone who betrayed my trust, and I'm trying to forgive and move forward. I'm seeking guidance and peace rather than staying in anger.", 'role': 'user'}, {'content': '{"issue_question": "I\'ve been hurt by someone who betrayed my trust, and I\'m trying to forgive and move forward. I\'m seeking guidance and peace rather than staying in anger.", "verse": "Matthew 6:14-15 - For if ye forgive men their trespasses, your heavenly Father will forgive you: But if ye forgive not men their trespasses, neither will your Father forgive your trespasses.", "explanation": "These verses teach that forgiveness is a choice we make, not merely a feeling that comes and goes. By choosing to forgive, we align with God’s mercy and open ourselves to healing, even when hurt remains. The passage also cautions that withholding forgiveness can hinder our own experience of divine mercy, reminding

## Get Hugging Face token

In [8]:
import sys
import os
from dotenv import load_dotenv

if 'google.colab' in sys.modules:
  from google.colab import userdata # type:ignore
  os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
  print("HF API Token set for Colab")
else:
  load_dotenv()
  print("Loaded env vars from .env")

HF API Token set for Colab


## Upload to Hugging Face

In [9]:
def upload_dataset(dataset, repo_name, token):
    dataset.push_to_hub(
        repo_name,
        token=token,
        private=False
    )

    print(f"Dataset uploaded successfully to: https://huggingface.co/datasets/{repo_name}")

upload_dataset(dataset, DATASET_REPO, os.environ.get("HF_TOKEN"))

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              :  70%|######9   |  524kB /  754kB            

...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              : 100%|##########| 13.5kB / 13.5kB            

                              : 100%|##########| 13.5kB / 13.5kB            

Dataset uploaded successfully to: https://huggingface.co/datasets/juvereturn/bible-dataset


## Create the dataset card

In [10]:
from huggingface_hub import DatasetCard

card_content = f"""---
---
pretty_name: "Test Dataset for CS-394"
license: mit
---

# Test Dataset

This is a test dataset of Python code snippets and explanations, used in DigiPen's CS-394 course.
"""

card = DatasetCard(card_content)
card.save(f"./README.md")

## Upload the dataset card

In [11]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_file(
    path_or_fileobj="./README.md",
    path_in_repo="README.md",
    repo_id=DATASET_REPO,
    repo_type="dataset",
)

CommitInfo(commit_url='https://huggingface.co/datasets/juvereturn/bible-dataset/commit/8f6264ba40563305e3f487777037adec5d7cf89a', commit_message='Upload README.md with huggingface_hub', commit_description='', oid='8f6264ba40563305e3f487777037adec5d7cf89a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/juvereturn/bible-dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='juvereturn/bible-dataset'), pr_revision=None, pr_num=None)